# Feels Protocol Simulation Calibration

This notebook calibrates the simulation parameters based on historical Solana CLMM data and generates baseline parameter configurations.

In [ ]:
import json
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
from pathlib import Path

# Import feels simulation components
import sys
sys.path.append('..')
from feels_sim.config import SimulationConfig
from feels_sim.participants import ParticipantConfig
from feels_sim.core import FeelsSimulation

## Historical Data Analysis (Placeholder)

In a production setup, this section would:
1. Pull historical swap data from Solana CLMMs (Orca Whirlpools, Raydium)
2. Analyze trade size distributions, frequency patterns, and liquidity utilization
3. Calibrate participant behavior parameters to match observed patterns

For now, we'll use reasonable baseline parameters based on DeFi market knowledge.

In [ ]:
# Placeholder for historical data analysis
# In production, this would pull from Solana RPC or data providers

def analyze_historical_trading_patterns():
    """Placeholder for historical data analysis."""
    # Mock data representing typical DeFi trading patterns
    return {
        'daily_volume_range': (50000, 500000),  # FeelsSOL
        'retail_trade_size_median': 100,
        'algo_trade_size_median': 2000,
        'trade_frequency_per_hour': 120,
        'lp_position_median': 10000,
        'fee_elasticity_coefficient': -1.5  # volume decreases as fees increase
    }

historical_patterns = analyze_historical_trading_patterns()
print("Historical trading patterns (mock data):")
for key, value in historical_patterns.items():
    print(f"  {key}: {value}")

## Baseline Parameter Configuration

Generate baseline parameters calibrated to typical DeFi market conditions.

In [ ]:
def create_baseline_config():
    """Create baseline simulation configuration."""
    
    # Market environment parameters
    sim_config = {
        'initial_sol_price_usd': 100.0,
        'sol_volatility_daily': 0.05,  # 5% daily volatility
        'sol_trend_bias': 0.0,  # No directional bias
        'base_fee_bps': 30,  # 0.30% base fee
        'buffer_share_pct': 85.0,
        'treasury_share_pct': 10.0,
        'creator_share_pct': 5.0,
        'jitosol_yield_apy': 0.07,  # 7% APY
        'enable_participant_behavior': True
    }
    
    # Participant behavior parameters (calibrated to DeFi patterns)
    participant_config = {
        # Retail traders: price-sensitive, sentiment-driven
        'retail_count': 80,
        'retail_base_frequency': 3.0,  # trades per day
        'retail_size_mean': 75.0,  # FeelsSOL
        'retail_size_variance': 1.0,
        'retail_fee_sensitivity': 2.5,  # high fee sensitivity
        'retail_sentiment_sensitivity': 0.8,
        
        # Algo traders: less fee sensitive, larger trades
        'algo_count': 15,
        'algo_base_frequency': 25.0,  # trades per day
        'algo_size_mean': 1500.0,  # FeelsSOL
        'algo_size_variance': 1.2,
        'algo_fee_sensitivity': 0.8,  # moderate fee sensitivity
        'algo_sentiment_sensitivity': 0.3,
        
        # Liquidity providers: fee-earnings focused
        'lp_count': 25,
        'lp_position_size_mean': 8000.0,  # FeelsSOL
        'lp_position_size_variance': 1.1,
        'lp_fee_sensitivity': 3.5,  # very sensitive to fee earnings
        'lp_rebalance_frequency': 8.0,  # hours
        
        # Arbitrageurs: opportunistic, fast execution
        'arb_count': 8,
        'arb_size_mean': 800.0,  # FeelsSOL
        'arb_threshold_bps': 8.0,  # 8 bps minimum profit
        'arb_response_time': 1.5  # minutes
    }
    
    return {
        'simulation_config': sim_config,
        'participant_config': participant_config,
        'calibration_metadata': {
            'created_date': '2024-10-16',
            'data_sources': ['mock_defi_patterns'],
            'version': '1.0.0',
            'description': 'Baseline configuration calibrated to typical DeFi market patterns'
        }
    }

baseline_params = create_baseline_config()
print("Baseline configuration created")
print(f"Retail traders: {baseline_params['participant_config']['retail_count']}")
print(f"Algo traders: {baseline_params['participant_config']['algo_count']}")
print(f"LPs: {baseline_params['participant_config']['lp_count']}")
print(f"Arbitrageurs: {baseline_params['participant_config']['arb_count']}")

## Parameter Validation

Test the calibrated parameters to ensure they produce reasonable market behavior.

In [ ]:
def validate_parameters(config_dict):
    """Validate calibrated parameters by running short simulations."""
    
    # Create configuration objects
    participant_config = ParticipantConfig(**config_dict['participant_config'])
    sim_config = SimulationConfig(
        participant_config=participant_config,
        **config_dict['simulation_config']
    )
    
    # Run validation simulation
    print("Running validation simulation (48 hours)...")
    sim = FeelsSimulation(sim_config)
    results = sim.run(hours=48)
    
    # Calculate key metrics
    total_volume = sum(h['total_volume'] for h in results.hourly_aggregates)
    avg_hourly_volume = total_volume / 48
    final_floor = results.snapshots[-1].floor_price_usd
    pomm_deployments = results.snapshots[-1].floor_state.pomm_deployments_count
    
    validation_results = {
        'total_volume_48h': total_volume,
        'avg_hourly_volume': avg_hourly_volume,
        'final_floor_price': final_floor,
        'pomm_deployments': pomm_deployments,
        'snapshots_count': len(results.snapshots)
    }
    
    print("Validation results:")
    for key, value in validation_results.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.2f}")
        else:
            print(f"  {key}: {value}")
    
    return validation_results

validation_results = validate_parameters(baseline_params)

# Check if results are reasonable
reasonable = (
    1000 < validation_results['total_volume_48h'] < 2000000 and  # Volume in reasonable range
    validation_results['final_floor_price'] > 0 and  # Floor price exists
    validation_results['pomm_deployments'] > 0  # POMM is deploying
)

print(f"\nParameters appear reasonable: {reasonable}")

## Fee Elasticity Testing

Test fee elasticity to ensure participant behavior responds appropriately to fee changes.

In [ ]:
def test_fee_elasticity(base_config):
    """Test fee elasticity across different fee levels."""
    
    fee_levels = [10, 20, 30, 40, 50]  # basis points
    volumes = []
    
    print("Testing fee elasticity...")
    
    for fee_bps in fee_levels:
        # Create config for this fee level
        config = base_config.copy()
        config['simulation_config']['base_fee_bps'] = fee_bps
        
        participant_config = ParticipantConfig(**config['participant_config'])
        sim_config = SimulationConfig(
            participant_config=participant_config,
            **config['simulation_config']
        )
        
        # Run short simulation
        sim = FeelsSimulation(sim_config)
        results = sim.run(hours=24)
        
        total_volume = sum(h['total_volume'] for h in results.hourly_aggregates)
        volumes.append(total_volume)
        
        print(f"  Fee {fee_bps}bps: Volume {total_volume:.0f}")
    
    # Calculate elasticity
    fee_changes = np.diff(fee_levels) / np.array(fee_levels[:-1])
    volume_changes = np.diff(volumes) / np.array(volumes[:-1])
    elasticity = volume_changes / fee_changes
    
    avg_elasticity = np.mean(elasticity)
    print(f"\nAverage fee elasticity: {avg_elasticity:.2f}")
    print(f"Volume decreases as fees increase: {volumes[-1] < volumes[0]}")
    
    return fee_levels, volumes, avg_elasticity

fee_levels, volumes, elasticity = test_fee_elasticity(baseline_params)

# Plot fee elasticity
plt.figure(figsize=(10, 6))
plt.plot(fee_levels, volumes, 'o-', linewidth=2, markersize=8)
plt.xlabel('Base Fee (basis points)')
plt.ylabel('24h Trading Volume (FeelsSOL)')
plt.title('Fee Elasticity: Volume vs Fee Level')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Fee elasticity validation: {'PASS' if elasticity < 0 else 'FAIL'} (should be negative)")

## Export Calibrated Parameters

Save the validated parameters to a configuration file for use in simulations.

In [ ]:
# Add validation results to metadata
baseline_params['calibration_metadata'].update({
    'validation_results': validation_results,
    'fee_elasticity': elasticity,
    'validation_passed': reasonable and elasticity < 0
})

# Save to experiments/configs/
output_path = Path('../experiments/configs/params_baseline.json')
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(baseline_params, f, indent=2)

print(f"Baseline parameters exported to: {output_path}")
print(f"Configuration includes {len(baseline_params)} sections")
print(f"Validation passed: {baseline_params['calibration_metadata']['validation_passed']}")

## Summary

This calibration notebook:

1. Analyzes historical trading patterns (placeholder implementation)
2. Creates baseline participant behavior parameters
3. Validates parameters through simulation testing
4. Tests fee elasticity to ensure realistic behavior
5. Exports calibrated parameters for simulation use

**Next Steps:**
- Replace placeholder historical analysis with real Solana CLMM data
- Implement additional market scenarios (bull/bear/sideways)
- Add LP retention and arbitrage efficiency metrics
- Create parameter sensitivity analysis